In [1]:
from random import randint, random
import collections
import os
import shutil

import numpy as np
import math
import time
from collections import deque
import torch as T
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import sklearn.neighbors
import sklearn.metrics
import sklearn.mixture
import matplotlib.pyplot as plt
import PIL
from nvidia.dali.plugin.pytorch import DALIGenericIterator

from imageComponent import *
from remote_read_write import scp_read_wrapper, scp_write_wrapper
scp_Tload = scp_read_wrapper(T.load, 'ly_1112103017@172.16.35.121', 30907, '/public/ly/zyz/cluster_id_rsa', '/public/ly/zyz/imageCL/tmp')
scp_Tsave = scp_write_wrapper(T.save, 'ly_1112103017@172.16.35.121', 30907, '/public/ly/zyz/cluster_id_rsa', '/public/ly/zyz/imageCL/tmp')

dali_device = 2
device = 'cuda:3'
T.backends.cudnn.benchmark = True
T.backends.cudnn.deterministic = True
use_amp = False

/public/ly/anaconda3/envs/zyz/lib/python3.10/site-packages/nvidia/dali/backend.py:46: Warning: DALI support for Python 3.10 is experimental and some functionalities may not work.
  deprecation_warning("DALI support for Python 3.10 is experimental and some functionalities "


In [2]:
size = (32, 32)
total = 100000000
batch_size = 25000

In [3]:
files = ['/public/ly/zyz/imageCL/101/0.998330k.png', '/public/ly/zyz/imageCL/101/0.998330v.png']

In [4]:
CLtrainpipe = DALIOverlapCLImageFolders(files=files, size=size, batch_size=batch_size,
                                        num_threads=1, device_id=dali_device)
CLtrainloader = dali.plugin.pytorch.DALIGenericIterator([CLtrainpipe], reader_name ='reader', output_map=['image1', 'image2', 'label'],
                                                    last_batch_policy=dali.plugin.base_iterator.LastBatchPolicy.FILL)

In [ ]:
overlap = 0
with T.inference_mode():
    for i in range(total // batch_size):
        for data in CLtrainloader:
            img1 = data[0]['image1']
            img2 = data[0]['image2']
            overlap += (img1 == img2).all(dim=-1).all(dim=-1).all(dim=-1).sum()
            #print((img1 == img2).all(dim=-1).all(dim=-1).all(dim=-1).shape)
            if overlap > 0:
                print(overlap)

  1%|█▏                                                                                                                                                | 33/4000 [01:29<3:06:24,  2.82s/it]